In [ ]:
from dataclasses import dataclass
from sika import Product, Provider, IntermediateTask

In [ ]:
def read_blueprint(*args):
    return "...", "Archimedes"

@dataclass(kw_only=True)
class Blueprint(Product):
    schema: str
    
class BlueprintReader(Provider[Blueprint]):
    @property
    def provided_parameters(self):
        return {"gizmo_name": []}
    
    def _call(self,parameters) -> Blueprint:
        gizmo_name = parameters["gizmo_name"]
        blueprint_loc = self.config['blueprint_location']

        schema, author = read_blueprint(gizmo_name,blueprint_loc)  # fictional function
        
        return Blueprint(
                    schema=schema,
                    parameters=parameters,  #  we don't have to store the params used to make it, but it can be useful
                    metadata={"blueprint_author":author}  # tack on extra information about this product
                )


In [ ]:
from sika.config import Config

config = Config("workshop_config.toml")

reader = BlueprintReader(config=config)
reader({"gizmo_name":"archimedian screw"})

In [ ]:
def parse_schema(schema):
    return []

def get_materials(*args, **kwargs):
    return "2C", ("wood",4,"silver","matte") 

@dataclass(kw_only=True)
class Parts(Product):
    frame: str
    n_screws: int
    wire: str
    paint: str
    
class PartFetcher(Provider[Parts], IntermediateTask[Provider[Blueprint]]):
    @property
    def provided_parameters(self):
        return {
                "wire_material": ["silver","copper","steel"],
            }
    
    def _call(self, parameters) -> Parts:
        wire_material = parameters["wire_material"]

        blueprint = self.prev(parameters) # call the previous task

        materials_list = parse_schema(blueprint.schema)
        parts_shelf, (frame, n_screws, wire, paint) = get_materials(materials_list, wire_material=wire_material)

        return Parts(
            frame=frame, n_screws=n_screws, wire=wire, paint=paint,
            metadata={"blueprint":blueprint, "parts_shelf": parts_shelf},
            parameters={"wire_material":wire_material}
        )

In [ ]:
reader = BlueprintReader()
fetcher = PartFetcher(prev=BlueprintReader())
fetcher.configure(config=config, logger=None) 

fetcher({"gizmo_name":"archimedian screw","wire_material":"silver"})